In [72]:
import os

# Torch
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
import torch.optim as optim
from torchvision.transforms import ToTensor, Compose, Normalize
from torchvision.datasets import MNIST
import torch.nn.functional as F
import torch.nn as nn
from PIL import Image

# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Dataset

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="WewILBCNfyY6D0eTbQ2j")
# project = rf.workspace("roboflow-universe-projects").project("fall-detection-ca3o8")
# version = project.version(4)
# dataset = version.download("yolov8")


In [73]:
# define dataset from yolov8 annotations
class CustomDataset(Dataset):
  def __init__(self, dir, transform=None, target_transform=None):
    self.img_dir = f"{dir}/images"
    self.label_dir = f"{dir}/labels"

    # extract file names from the directory, removing the file extension and parent directory
    self.img_names = [os.path.splitext(name)[0] for name in os.listdir(self.img_dir)]

    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_names)

  def __getitem__(self, idx):
    # construct the file paths for the image and label from the directory, add extension
    img_path = os.path.join(self.img_dir, self.img_names[idx] + ".jpg")
    label_path = os.path.join(self.label_dir, self.img_names[idx] + ".txt")
    image = Image.open(img_path)
    labels = [[float(l) for l in line.rstrip('\n').split()] for line in open(label_path)]
    if self.transform:
      image = self.transform(image)  
    if self.target_transform:
      labels = self.target_transform(labels)
    return image, labels

In [76]:
dataset = CustomDataset("Fall-Detection-4/train", transform=ToTensor())

In [42]:
len(dataset)

9438

In [40]:
dataset[0][0].shape

torch.Size([3, 640, 640])

In [78]:
# # compute mean and standard deviation of the dataset
# concat = torch.cat([dataset[i][0] for i in range(4000)], 0).to(torch.float32)
# mean = concat.mean()
# std = concat.std()
# mean, std

(tensor(0.4379), tensor(0.3040))

In [80]:
transform_data = Compose([ToTensor(), Normalize((0.4379,), (0.3040,))])

batch_size = 256
train_dataset = CustomDataset("Fall-Detection-4/train", transform=transform_data)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [81]:
# define the object classification and bounding box regression model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def train(model, train_loader, epochs, criterion):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 10 == 9:
                print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 10}")
                running_loss = 0.0
    print("Finished Training")

In [ ]:
model = Net().to(device)